In [1]:
import sys
src_path = "/Users/kimathikaai/workspace/fond/"
if src_path not in sys.path:
    sys.path.insert(0, src_path)
print(sys.path)

['/Users/kimathikaai/workspace/fond/', '/Users/kimathikaai/workspace/fond/src/utils', '/opt/homebrew/Cellar/python@3.11/3.11.4_1/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.4_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.4_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/kimathikaai/workspace/envs/fond/lib/python3.11/site-packages']


In [2]:
import argparse

import wandb
import yaml
import copy
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.train.fit import fit
from src.utils.hparams import random_hparams, seed_hash
from src.utils.run_info import get_project_runs, find_best_steps

%matplotlib inline
%load_ext autoreload
%autoreload 2

ENTITY = 'critical-ml-dg'
PROJECT_NAME = 'classes'
# SWEEP_IDS = ['518qlq2m', '0alcovf9', '2suqerrf', '5gjc78bl']
SWEEP_IDS = ['rl20kabz', 'd6y5q7ul']
# FILTERING_CRITERIA = {'algorithm':'ERM'}
FILTERING_CRITERIA = {}
METRIC_NAME = 'val/nacc'
TEST_METRIC = 'test/nacc'
METRIC_GOAL = 'max'

In [3]:
# setup api
api = wandb.Api(
    overrides={
        "entity": ENTITY,
    },
    timeout=29
)

# get runs
runs, unique_datasets = get_project_runs(
    api_conn=api,
    project_name=PROJECT_NAME,
    sweep_ids=SWEEP_IDS,
    filtering_criteria=FILTERING_CRITERIA,
)

[info] Extracting sweep information from the following sweeps: ['rl20kabz', 'd6y5q7ul']


100%|██████████████████████████████████████████████████████| 120/120 [00:01<00:00, 65.72it/s]

[info] Runs to process: 180


In [4]:
# visualize a run
# type(runs[0])
# hist = list(runs[0].scan_history(keys=['val/nacc', 'step']))
runs[0]

<Run critical-ml-dg/classes/dxtzcjy4 (finished)>

In [5]:
# visualize a run metric
list(runs[0].scan_history(keys=['val/nacc', 'step']))[-1]

{'val/nacc': 0.2873176336288452, 'step': 5000}

In [6]:
best_step_info = find_best_steps(
    runs=runs, 
    val_metric=METRIC_NAME, 
    metric_goal=METRIC_GOAL, 
    test_metric=TEST_METRIC
)

100%|██████████████████████████████████████████████████████| 180/180 [15:06<00:00,  5.04s/it]


In [7]:
key = list(best_step_info.keys())[0]
key, best_step_info[key]

(<Run critical-ml-dg/classes/dxtzcjy4 (finished)>,
 {'val/nacc': 0.2989225586255391,
  'step_number': 4500,
  'test/nacc': 0.5187258124351501})

In [10]:
# Runs are organized based on dataset, overlap and test_id
# dataset -> num_domain_linked_classes -> num_classes -> test_domain_id -> [hparam_id]
# Note: Performing only one trial
INFO = {
    dataset:{
        num_domain_linked_classes:{
            num_classes:{
                test_domain_id: {
                    'run_id': None, 
                    'step': None,
                    METRIC_NAME: float("inf") if METRIC_GOAL == "min" else float("-inf"),
                    TEST_METRIC: None
                } for test_domain_id in range(4)
            } for num_classes in range(5,66)
        } for num_domain_linked_classes in range(5,70,5)
    } 
    for dataset in ['PACS', 'VLCS', 'OfficeHome']
}
algorithm_info = {}
for run, info in tqdm(best_step_info.items()):
    # Get run infomation
    run_id = run.id
    run_dataset = run.config["dataset"]
    run_test_domain_id = run.config["test_set_id"]
    run_num_domain_linked_classes = run.config['num_domain_linked_classes']
    run_num_classes = run.config['num_classes']
    run_algorithm = run.config['algo']
    # get historical data
    run_val = info[METRIC_NAME]
    run_step = info['step_number']
    run_test = info[TEST_METRIC]

    # create entry 
    if run_algorithm not in algorithm_info:
        algorithm_info[run_algorithm] = copy.deepcopy(INFO)
        algorithm_info[
            run_algorithm][
            run_dataset][
            run_num_domain_linked_classes][
            run_num_classes][
            run_test_domain_id] = {
                "run_id": run_id, 
                METRIC_NAME: run_val, 
                'step': run_step,
                TEST_METRIC: run_test
            }
        # print(algorithm_info)
        continue

    # get previous validation value
    previous_val = algorithm_info[
            run_algorithm][
            run_dataset][
            run_num_domain_linked_classes][
            run_num_classes][
            run_test_domain_id][METRIC_NAME]

    # compare based on goal
    if (METRIC_GOAL=="min") and (previous_val>run_val):
        algorithm_info[
            run_algorithm][
            run_dataset][
            run_num_domain_linked_classes][
            run_num_classes][
            run_test_domain_id] = {
                "run_id": run_id, 
                METRIC_NAME: run_val, 
                'step': run_step,
                TEST_METRIC: run_test
            }
    elif (METRIC_GOAL=="max") and (previous_val<run_val):
        algorithm_info[
            run_algorithm][
            run_dataset][
            run_num_domain_linked_classes][
            run_num_classes][
            run_test_domain_id] = {
                "run_id": run_id, 
                METRIC_NAME: run_val, 
                'step': run_step,
                TEST_METRIC: run_test
            }

algorithm_info    

100%|████████████████████████████████████████████████████| 180/180 [00:00<00:00, 3003.46it/s]


{'ERM': {'PACS': {5: {5: {0: {'run_id': None,
      'step': None,
      'val/nacc': -inf,
      'test/nacc': None},
     1: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None},
     2: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None},
     3: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None}},
    6: {0: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None},
     1: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None},
     2: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None},
     3: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None}},
    7: {0: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None},
     1: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None},
     2: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None},
     3: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None}},
    8: {0: {'run_id': None, 

In [ ]:
algorithm_info['ERM']['OfficeHome'][5][5]

In [11]:
# CREATE TABLE
# For a given algorithm and dataset create a table
# Create a table -> algorithm, dataset, num_domain_linked_classes, num_classes, test_domain_id, test/nacc
# Note we will need to average
table = []
for algo in algorithm_info:
    for dataset in algorithm_info[algo]:
        for num_domain_linked_classes in algorithm_info[algo][dataset]:
            for num_classes in algorithm_info[algo][dataset][num_domain_linked_classes]:
                for test_domain_id in algorithm_info[algo][dataset][num_domain_linked_classes][num_classes]:
                    row = {
                        'algorithm': algo,
                        'dataset': dataset,
                        'num_domain_linked_classes': num_domain_linked_classes,
                        'num_domain_shared_classes': num_classes - num_domain_linked_classes,
                        'num_classes': num_classes,
                        'test_domain_id': test_domain_id,
                        TEST_METRIC: algorithm_info[algo][dataset][num_domain_linked_classes][num_classes][test_domain_id][TEST_METRIC]
                    }
                    table.append(row)
df = pd.DataFrame(table)
df

,algorithm,dataset,num_domain_linked_classes,num_domain_shared_classes,num_classes,test_domain_id,test/nacc
0,ERM,PACS,5,0,5,0,NaN
1,ERM,PACS,5,0,5,1,NaN
2,ERM,PACS,5,0,5,2,NaN
3,ERM,PACS,5,0,5,3,NaN
4,ERM,PACS,5,1,6,0,NaN
...,...,...,...,...,...,...,...
19027,FOND,OfficeHome,65,-1,64,3,NaN
19028,FOND,OfficeHome,65,0,65,0,NaN
19029,FOND,OfficeHome,65,0,65,1,NaN
19030,FOND,OfficeHome,65,0,65,2,NaN


In [12]:
df.to_csv('2024-02-29.csv', index=False)

In [14]:
df = pd.read_csv('2024-02-29.csv')
_df = df.groupby(['algorithm', 'dataset', 'num_domain_linked_classes', 'num_domain_shared_classes', 'num_classes']).mean(numeric_only=True)
_df.loc[_df[TEST_METRIC].notnull()]

test_domain_id  \
algorithm dataset    num_domain_linked_classes num_domain_shared_classes num_classes                   
ERM       OfficeHome 5                         0                         5                       1.5   
                                               1                         6                       1.5   
                                               2                         7                       1.5   
                     15                        0                         15                      1.5   
                                               20                        35                      1.5   
                                               50                        65                      1.5   
FOND      OfficeHome 5                         0                         5                       1.5   
                                               1                         6                       1.5   
                                               2                         7                       1.5   

                                                                                      test/nacc  
algorithm dataset    num_domain_linked_classes num_domain_shared_classes num_classes             
ERM       OfficeHome 5                         0                         5             0.825242  
                                               1                         6             0.761654  
                                               2                         7             0.696732  
                     15                        0                         15            0.708261  
                                               20                        35            0.517886  
                                               50                        65            0.394472  
FOND      OfficeHome 5                         0                         5             0.824579  
                                               1                         6             0.751962  
                                               2                         7             0.716681